# Recomendación de Animes basado en Similaridad de Texto y KNN

Este Colab implementa un sistema de recomendación de animes utilizando técnicas de preprocesamiento de texto, codificación de variables categóricas, y un modelo de vecinos más cercanos (KNN) para encontrar animes similares. El proceso completo incluye la carga y limpieza de datos, la transformación de características, y la construcción del modelo de recomendación.

## Importación de Librerías

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.neighbors import NearestNeighbors
import random
import joblib

## Carga de Datos:

Se carga un archivo CSV que contiene los datos de los animes.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/tfm/scraping_animeList.csv')

## Exploración y Limpieza de Datos:

- Se verifican las dimensiones del DataFrame y se observan los primeros registros.
- Se identifican las columnas con valores faltantes y se eliminan aquellas con muchos valores faltantes y posibles errores.
- Se eliminan filas con valores faltantes restantes.

In [ ]:
# Total de filas y columnas
df.shape

(12598, 20)

In [ ]:
df.head()

,ID,Title,Type,Episodes,Source,Studios,Demographic,Year,Producers,Genres,Licensors,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,Image_URL,Synopsis
0,1,Cowboy Bebop,TV,26.0,Original,Sunrise,NaN,1998,Bandai Visual,"Action, Award Winning, Sci-Fi",Funimation,24,R - 17+ (violence & profanity),8.0,46.0,43,"1,864,022","82,369",https://cdn.myanimelist.net/images/anime/4/196...,"Crime is timeless. By the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,1.0,Original,Bones,NaN,2001,"Sunrise, Bandai Visual","Action, Sci-Fi","Sony Pictures Entertainment, Funimation",1,R - 17+ (violence & profanity),8.0,193.0,619,"377,963","1,581",https://cdn.myanimelist.net/images/anime/1439/...,"Another day, another bounty—such is the life o..."
2,6,Trigun,TV,26.0,Manga,Madhouse,Shounen,1998,Victor Entertainment,"Action, Adventure, Sci-Fi",Funimation,24,PG-13 - Teens 13 or older,8.0,341.0,252,"763,023","16,009",https://cdn.myanimelist.net/images/anime/7/203...,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,TV,26.0,Original,Sunrise,NaN,2002,"Bandai Visual, Dentsu, Victor Entertainment, T...","Action, Drama, Mystery, Supernatural","Funimation, Bandai Entertainment",25,PG-13 - Teens 13 or older,7.0,3073.0,1857,"116,691",648,https://cdn.myanimelist.net/images/anime/10/19...,Robin Sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,TV,52.0,Manga,Toei Animation,Shounen,2004,"TV Tokyo, Dentsu","Action, Adventure, Fantasy",Illumitoon Entertainment,23,PG - Children,6.0,4542.0,5354,"15,526",15,https://cdn.myanimelist.net/images/anime/7/215...,It is the dark century and the people are suff...


In [ ]:
# Comprobamos columnas con valores faltantes
df.isna().mean()*100

ID              0.000000
Title           0.000000
Type            9.176060
Episodes        0.547706
Source          0.000000
Studios         0.000000
Demographic    66.796317
Year            0.000000
Producers       0.000000
Genres         10.501667
Licensors      68.566439
Duration        0.000000
Rating          1.270043
Score           0.000000
Ranked         18.280679
Popularity      0.000000
Members         0.000000
Favorites       0.000000
Image_URL       0.000000
Synopsis        0.023813
dtype: float64

In [ ]:
# Eliminar las columnas con mayor número de valores faltantes y las IDs además eliminamos Studios y Producers ya que tienen muchos 'add some' (posible error al scrapear)
df.drop(['ID', 'Demographic', 'Licensors', 'Studios', 'Producers'], axis=1, inplace=True)

In [ ]:
# Comprobamos cuantas filas con valores faltantes obtenemos
sum(df.isna().any(axis=1))

3056

In [ ]:
# Procedemos a eliminarlas
df.dropna(axis=0, inplace=True)

In [ ]:
df.shape

(9542, 15)

## Transformación de Datos:

- Se convierten las columnas Episodes, Score y Ranked a enteros.
- Se eliminan las comas de las columnas Members y Favorites y se convierten a enteros.
- Se aplican LabelEncoder a las columnas categóricas Type, Source, y Rating.
- Se codifican los géneros utilizando one-hot encoding y se combinan con el DataFrame original.

In [ ]:
# Convertimos a enteros los float
df.Episodes = df.Episodes.astype(int)
df.Score = df.Score.astype(int)
df.Ranked = df.Ranked.astype(int)

In [ ]:
# Reemplazamos la coma por nada y convertimos a enteros
df.Members = df.Members.str.replace(',','').astype(int)
df.Favorites = df.Favorites.str.replace(',','').astype(int)

In [ ]:
# Aplicamos Label Encoder
cols = ['Type', 'Source', 'Rating']
le = LabelEncoder()
for col in cols:
  df[col] = le.fit_transform(df[col])

In [ ]:
# Codificamos los géneros con 1-hot encoding
df_genres = df.Genres.str.get_dummies(sep=',')

In [ ]:
# Unimos los géneros codificados al df
df_num = pd.concat([df.drop('Genres', axis=1), df_genres], axis=1)

In [ ]:
# Filtramos de modo que nos quedamos con aquellas series que tienen sinopsis ya que la usaremos para medir la similitud
df_num = df_num[~df_num['Synopsis'].str.contains('No synopsis information')]

## Preprocesamiento de Texto:

- Se filtran las series que tienen sinopsis, eliminando aquellas sin información.
- Se preprocesa el texto de las sinopsis eliminando caracteres no alfabéticos y espacios adicionales.
- Se convierte el texto en vectores TF-IDF.

In [ ]:
# Preprocesamos el texto
def preprocess_text(text):
  # Dejamos palabras
  text = ' '.join(re.findall(r'\b[a-zA-Z]+\b', text))
  # Eliminación de los espacios adicionales
  text = re.sub(r'\s+', ' ', text).strip()
  return text.lower()
df_num['Synopsis'] = df_num['Synopsis'].apply(preprocess_text)

In [ ]:
# Codificamos el texto a formato numérico con tfidf
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=0.001)
X_tfidf = vectorizer.fit_transform(df_num['Synopsis'])

## Escalado y Combinación de Datos:

- Se escalan las características numéricas.
- Se combinan los datos escalados con los vectores TF-IDF.

In [ ]:
# Seleccionar las numéricas del df y escalarlas
X_num = df_num.drop(['Title', 'Image_URL', 'Synopsis'], axis=1)
X_num_sc = MinMaxScaler().fit_transform(X_num)

In [ ]:
# Unir horizontalmente la matriz de tfidf y de los datos escalados
X = np.hstack((X_num_sc, X_tfidf.toarray()))
X.shape

(9062, 5373)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Crear un vectorizador CountVectorizer
count_vectorizer = CountVectorizer()

# Entrenar el vectorizador con los títulos de los animes
count_vectorizer.fit(df_num['Title'])

CountVectorizer()

## Entrenamiento del Modelo de Recomendación:

- Se entrena un modelo KNN utilizando la métrica de distancia coseno.

In [ ]:
# Entrenamos un modelo KNN para obtener las distancias entre series
model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine").fit(X)

In [ ]:
# Guardar el modelo KNN y el vectorizador TF-IDF en archivos utilizando joblib
joblib.dump(count_vectorizer, '/content/drive/MyDrive/dataset/tfm/vectorizer2.pkl')
joblib.dump(model, '/content/drive/MyDrive/dataset/tfm/modelo_knn2.pkl')

['/content/drive/MyDrive/dataset/tfm/vectorizer.pkl']

In [ ]:
# Reseteamos el indice
df_num.reset_index(inplace=True)

## Funciones de Recomendación:

- Se define una función para obtener recomendaciones basadas en la similitud de títulos y sinopsis.

In [ ]:
# Función de recomendación de series
def get_anime_recommendations(title):
    movie_index = df_num[df_num["Title"] == title].index[0]
    distances, indices = model.kneighbors(np.array([X[movie_index]]))
    similar_movies = [(df_num["Title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]
get_anime_recommendations('Cowboy Bebop')

[('Code Geass: Hangyaku no Lelouch R2', 0.18510794749933235),
 ('Code Geass: Hangyaku no Lelouch', 0.18610596407991742),
 ('Tengen Toppa Gurren Lagann', 0.18752404802119038),
 ('Koukaku Kidoutai: Stand Alone Complex', 0.21540994086582532),
 ('Macross F', 0.22484044811496062)]

In [ ]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9062 entries, 0 to 9061
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           9062 non-null   int64 
 1   Title           9062 non-null   object
 2   Type            9062 non-null   int64 
 3   Episodes        9062 non-null   int64 
 4   Source          9062 non-null   int64 
 5   Year            9062 non-null   int64 
 6   Duration        9062 non-null   int64 
 7   Rating          9062 non-null   int64 
 8   Score           9062 non-null   int64 
 9   Ranked          9062 non-null   int64 
 10  Popularity      9062 non-null   int64 
 11  Members         9062 non-null   int64 
 12  Favorites       9062 non-null   int64 
 13  Image_URL       9062 non-null   object
 14  Synopsis        9062 non-null   object
 15   Adventure      9062 non-null   int64 
 16   Avant Garde    9062 non-null   int64 
 17   Award Winning  9062 non-null   int64 
 18   Boys Lo